In [70]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
# import findspark
# import os
# spark_location='/opt/spark-2.4.3/' # Set your own
# java8_location= '/usr/lib/jvm/java-8-openjdk-amd64' # Set your own
# os.environ['JAVA_HOME'] = java8_location
# findspark.init(spark_home=spark_location) 

In [3]:
sc = pyspark.SparkContext()

In [4]:
from pyspark.sql import SQLContext

In [5]:
sqlContext = SQLContext(sc)

In [6]:
import os
os.getcwd()

'C:\\Users\\A.Ram\\Desktop\\BI_Work\\parkash_work'

In [7]:
print(os.environ['SPARK_HOME'])
print(os.environ['JAVA_HOME'])
print(os.environ['PATH']) 

C:\Users\A.Ram\spark\spark-2.4.3-bin-hadoop2.7
C:\Program Files\Java\jdk1.8.0_221
C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\WINDOWS\system32;C:\WINDOWS;C:\WINDOWS\System32\Wbem;C:\WINDOWS\System32\WindowsPowerShell\v1.0\;C:\Program Files\Intel\WiFi\bin\;C:\Program Files\Common Files\Intel\WirelessCommon\;C:\Program Files (x86)\Common Files\Hitachi ID\;C:\Program Files\Common Files\Hitachi ID\;C:\Users\.ARam\AppData\Local\pip;C:\Users\A.Ram\AppData\Local\Programs\Python\Python37-32\Scripts;C:\Program Files\TortoiseGit\bin;C:\Program Files\Microsoft SQL Server\Client SDK\ODBC\130\Tools\Binn\;C:\Program Files (x86)\Microsoft SQL Server\140\Tools\Binn\;C:\Program Files\Microsoft SQL Server\140\Tools\Binn\;C:\Program Files\Microsoft SQL Server\140\DTS\Binn\;C:\Program Files (x86)\Microsoft SQL Server\150\DTS\Binn\;C:\WINDOWS\System32\OpenSSH\;C:\Users\A.Ram\AppData\Local\Continuum\anaconda3;C:\Users\A.Ram\AppData\Local\Continuum\anaconda3\Library\mingw-w64\bin;C:\Users\A.R

In [81]:
df = sqlContext.read.format("csv").option("header", "true").load("../2019*.csv")

In [82]:
df

DataFrame[MATERIAL: string, PLNT: string, BATCH: string, UNRESTRICTED: string, TRANSIT/TRANSF.: string, IN QUALITY INSP.: string, RESTRICTED-USE: string, BLOCKED: string, RETURNS: string, STATUS_DATE: string]

In [83]:
df.columns

['MATERIAL',
 'PLNT',
 'BATCH',
 'UNRESTRICTED',
 'TRANSIT/TRANSF.',
 'IN QUALITY INSP.',
 'RESTRICTED-USE',
 'BLOCKED',
 'RETURNS',
 'STATUS_DATE']

In [84]:
df = df.withColumnRenamed('UNRESTRICTED',"st1").withColumnRenamed("TRANSIT/TRANSF.","st2").withColumnRenamed("MATERIAL",'sku_code').withColumnRenamed("PLNT","sc_node_code").withColumnRenamed("STATUS_DATE","status_date").withColumnRenamed("BATCH","batch")

In [85]:
from pyspark.sql.types import DoubleType

In [86]:
df = df.withColumnRenamed('IN QUALITY INSP.',"IN_QUALITY_INSP")

In [87]:
df

DataFrame[sku_code: string, sc_node_code: string, batch: string, st1: string, st2: string, IN_QUALITY_INSP: string, RESTRICTED-USE: string, BLOCKED: string, RETURNS: string, status_date: string]

In [88]:
df.columns

['sku_code',
 'sc_node_code',
 'batch',
 'st1',
 'st2',
 'IN_QUALITY_INSP',
 'RESTRICTED-USE',
 'BLOCKED',
 'RETURNS',
 'status_date']

In [89]:
df = df.withColumn('st3', df['IN_QUALITY_INSP'] + df['RESTRICTED-USE'] + df['BLOCKED'] + df['RETURNS'])

In [90]:
cols_to_drop = ['IN_QUALITY_INSP','RESTRICTED-USE','BLOCKED','RETURNS']
df = df.drop(*cols_to_drop)

In [91]:
df

DataFrame[sku_code: string, sc_node_code: string, batch: string, st1: string, st2: string, status_date: string, st3: double]

In [92]:
df = df.withColumn("st1", df["st1"].cast(DoubleType()))

In [93]:
df = df.withColumn("st2", df["st2"].cast(DoubleType()))

In [94]:
df.dtypes

[('sku_code', 'string'),
 ('sc_node_code', 'string'),
 ('batch', 'string'),
 ('st1', 'double'),
 ('st2', 'double'),
 ('status_date', 'string'),
 ('st3', 'double')]

In [1]:
from pyspark.sql.functions import substring, length, col, expr

In [2]:
?substring

In [108]:
from pyspark.sql.functions import substring, length, col, expr
df=df.withColumn('length_check', length('sku_code'))
# df.withColumn('b', df.batch.substring(1, 3))
df=df.withColumn("batch_check", substring(df.batch,-1,2))
df.show()

+-------------+------------+----------+------+----+-----------+---+------------+-----------+
|     sku_code|sc_node_code|     batch|   st1| st2|status_date|st3|length_check|batch_check|
+-------------+------------+----------+------+----+-----------+---+------------+-----------+
|RL10J0AS10AH1|        1002|LIAW      |2136.0| 0.0| 2019-06-22|0.0|          13|           |
|RL10J0AS10AH1|        AP01|LIAW      |   4.0| 0.0| 2019-06-22|0.0|          13|           |
|RL10J0AS10AH1|        AP02|LIAW      |   2.0| 0.0| 2019-06-22|0.0|          13|           |
|RL10J0AS10AH1|        AP05|LIAW      |   2.0| 0.0| 2019-06-22|0.0|          13|           |
|RL10J0AS10AH1|        AP06|LIAW      |   2.0| 0.0| 2019-06-22|0.0|          13|           |
|RL10J0AS10AH1|        AP10|LIAW      |   2.0| 0.0| 2019-06-22|0.0|          13|           |
|RL10J0AS10AH1|        AP12|LIAW      |   2.0| 0.0| 2019-06-22|0.0|          13|           |
|RL10J0AS10AH1|        AS01|LIAW      |  30.0| 0.0| 2019-06-22|0.0|   

In [23]:
from pyspark.sql import functions as F
df = df.withColumn(
    'sales_channel_code',
    F.when((F.col("batch_check") == 'AW'),'REP')\
    .when((F.col("batch_check") == 'KZ'),'REP')\
    .when((F.col("batch_check") == 'CC'),'REP')\
    .when((F.col("batch_check") == 'DF'),'OEM')\
    .when((F.col("batch_check") == 'ST'),'OEM')\
    .when((F.col("batch_check") == 'OE'),'OEM')\
    .when((F.col("batch_check") == 'EX'),'EX')\
)

In [24]:
df = df.dropna(subset=('sales_channel_code'))

In [25]:
df = df.selectExpr("status_date","sku_code","sc_node_code","sales_channel_code",
              "stack(3, 'st1', st1, 'st2', st2, 'st3' ,st3) as (stock_type, stock_qty)")

In [26]:
df

DataFrame[status_date: string, sku_code: string, sc_node_code: string, sales_channel_code: string, stock_type: string, stock_qty: double]

In [27]:
df.show()

+-----------+-------------+------------+------------------+----------+---------+
|status_date|     sku_code|sc_node_code|sales_channel_code|stock_type|stock_qty|
+-----------+-------------+------------+------------------+----------+---------+
| 2019-06-24|RL10J0AS10AH1|        1002|               REP|       st1|     80.0|
| 2019-06-24|RL10J0AS10AH1|        1002|               REP|       st2|      0.0|
| 2019-06-24|RL10J0AS10AH1|        1002|               REP|       st3|      0.0|
| 2019-06-24|RL10J0AS10AH1|        1002|               REP|       st1|   2036.0|
| 2019-06-24|RL10J0AS10AH1|        1002|               REP|       st2|      0.0|
| 2019-06-24|RL10J0AS10AH1|        1002|               REP|       st3|      0.0|
| 2019-06-24|RL10J0AS10AH1|        AP01|               REP|       st1|      3.0|
| 2019-06-24|RL10J0AS10AH1|        AP01|               REP|       st2|      0.0|
| 2019-06-24|RL10J0AS10AH1|        AP01|               REP|       st3|      0.0|
| 2019-06-24|RL10J0AS10AH1| 

In [28]:
forecast_df = sqlContext.read.format("csv").option("header", "true").load("../forecast_june.csv")

In [29]:
forecast_df.show()

+----------+---------+--------+--------+-------+-------------+------------+------------------+------------+----------------+
|source_tag|for_month|for_year|in_month|in_year|     sku_code|sc_node_code|sales_channel_code|forecast_qty|  forecast_value|
+----------+---------+--------+--------+-------+-------------+------------+------------------+------------+----------------+
|         1|        6|    2019|       6|   2019|RL10J0AS10AH1|        1002|               REP|   1946.0000|4378500.00000000|
|         1|        6|    2019|       6|   2019|RL10J0AS10AH1|        AP01|               REP|    300.0000| 675000.00000000|
|         1|        6|    2019|       6|   2019|RL10J0AS10AH1|        AP02|               REP|     15.0000|  33750.00000000|
|         1|        6|    2019|       6|   2019|RL10J0AS10AH1|        AP03|               REP|     30.0000|  67500.00000000|
|         1|        6|    2019|       6|   2019|RL10J0AS10AH1|        AP04|               REP|    200.0000| 450000.00000000|


In [30]:
calendar_df = sqlContext.read.format("csv").option("header", "true").load("../calendar.csv")

In [31]:
calendar_df = calendar_df[['for_date','for_month','for_year','days_in_month']]

In [32]:
calendar_df.dtypes

[('for_date', 'string'),
 ('for_month', 'string'),
 ('for_year', 'string'),
 ('days_in_month', 'string')]

In [33]:
calendar_df.show()

+----------+---------+--------+-------------+
|  for_date|for_month|for_year|days_in_month|
+----------+---------+--------+-------------+
|29-01-2013|        1|    2013|           31|
|25-01-2013|        1|    2013|           31|
|28-01-2013|        1|    2013|           31|
|27-01-2013|        1|    2013|           31|
|26-01-2013|        1|    2013|           31|
|30-01-2013|        1|    2013|           31|
|31-01-2013|        1|    2013|           31|
|08-02-2013|        2|    2013|           28|
|13-02-2013|        2|    2013|           28|
|28-02-2013|        2|    2013|           28|
|27-02-2013|        2|    2013|           28|
|26-04-2013|        4|    2013|           30|
|25-04-2013|        4|    2013|           30|
|21-06-2013|        6|    2013|           30|
|29-07-2013|        7|    2013|           31|
|26-07-2013|        7|    2013|           31|
|09-02-2013|        2|    2013|           28|
|11-02-2013|        2|    2013|           28|
|12-02-2013|        2|    2013|   

In [34]:
from pyspark.sql.types import DateType
from pyspark.sql.functions import col, unix_timestamp, to_date
calendar_df = calendar_df.withColumn('for_date', 
                   to_date(unix_timestamp(col('for_date'), 'dd-MM-yyyy').cast("timestamp")))
# df.show()
# df.printSchema()
# # calendar_df = calendar_df.withColumn("for_date", calendar_df["for_date"].cast(DateType()))
# # val df = sc.parallelize(Seq("2016-08-26")).toDF("Id")
# # val df2 = df.withColumn("Timestamp", (col("Id").cast("timestamp")))
# calendar_df = calendar_df.withColumn("Date", (calendar_df["for_date"].cast("timestamp")))

In [35]:
calendar_df.show()

+----------+---------+--------+-------------+
|  for_date|for_month|for_year|days_in_month|
+----------+---------+--------+-------------+
|2013-01-29|        1|    2013|           31|
|2013-01-25|        1|    2013|           31|
|2013-01-28|        1|    2013|           31|
|2013-01-27|        1|    2013|           31|
|2013-01-26|        1|    2013|           31|
|2013-01-30|        1|    2013|           31|
|2013-01-31|        1|    2013|           31|
|2013-02-08|        2|    2013|           28|
|2013-02-13|        2|    2013|           28|
|2013-02-28|        2|    2013|           28|
|2013-02-27|        2|    2013|           28|
|2013-04-26|        4|    2013|           30|
|2013-04-25|        4|    2013|           30|
|2013-06-21|        6|    2013|           30|
|2013-07-29|        7|    2013|           31|
|2013-07-26|        7|    2013|           31|
|2013-02-09|        2|    2013|           28|
|2013-02-11|        2|    2013|           28|
|2013-02-12|        2|    2013|   

In [36]:
from pyspark.sql.functions import unix_timestamp, lit
from datetime import datetime
date_start = '2019-06-01'
date_end ='2019-06-30'
date_start = datetime.strptime(str(date_start),'%Y-%m-%d')
date_end = datetime.strptime(str(date_end),'%Y-%m-%d')

calendar_df=calendar_df.filter(calendar_df["for_date"]>=date_start).filter(calendar_df["for_date"]<=date_end)

# .filter(calendar_df["for_date"].lt(lit(date_end)))     

# dates = ("2019-06-01",  "2019-06-30")
# from pyspark.sql.functions import col
# calendar_df.where(col('for_date').between(*dates)).show(truncate=False)

# calendar_df=calendar_df.withColumn("for_date", to_date(unix_timestamp(calendar_df["for_date"], "MM/dd/yyyy").cast("timestamp")))
# calendar_df.filter(calendar_df["for_date"] >= lit('2017-01-01')) \
#        .filter(calendar_df["for_date"] <= lit('2017-01-31')).show()

In [37]:
calendar_df.show()

+----------+---------+--------+-------------+
|  for_date|for_month|for_year|days_in_month|
+----------+---------+--------+-------------+
|2019-06-02|        6|    2019|           30|
|2019-06-04|        6|    2019|           30|
|2019-06-03|        6|    2019|           30|
|2019-06-01|        6|    2019|           30|
|2019-06-07|        6|    2019|           30|
|2019-06-05|        6|    2019|           30|
|2019-06-06|        6|    2019|           30|
|2019-06-09|        6|    2019|           30|
|2019-06-08|        6|    2019|           30|
|2019-06-10|        6|    2019|           30|
|2019-06-17|        6|    2019|           30|
|2019-06-30|        6|    2019|           30|
|2019-06-24|        6|    2019|           30|
|2019-06-29|        6|    2019|           30|
|2019-06-25|        6|    2019|           30|
|2019-06-23|        6|    2019|           30|
|2019-06-27|        6|    2019|           30|
|2019-06-26|        6|    2019|           30|
|2019-06-28|        6|    2019|   

In [38]:
calendar_df

DataFrame[for_date: date, for_month: string, for_year: string, days_in_month: string]

In [41]:
products_df = forecast_df[['sku_code','sc_node_code','sales_channel_code','for_month','for_year']]
products_df=products_df.dropDuplicates()
# products_df = forecast_df.dropDuplicates(['sku_code','sc_node_code','sales_channel_code','for_month','for_year'])


In [42]:
products_df.show()

+-------------+------------+------------------+---------+--------+
|     sku_code|sc_node_code|sales_channel_code|for_month|for_year|
+-------------+------------+------------------+---------+--------+
|RL1AK0AH10AW1|        GJ09|               REP|        6|    2019|
|RL1AL0AH10AW1|        GJ02|               REP|        6|    2019|
|RLBJKHEMH3AM1|        KL02|               REP|        6|    2019|
|RLBJKHEMH3AM1|        MA03|               REP|        6|    2019|
|RLBJKHERD1AL2|        AP10|               REP|        6|    2019|
|RLBJKHERD1AL2|        GJ03|               REP|        6|    2019|
|RLBJKHETA3AL1|        JK01|               REP|        6|    2019|
|RLBJKHETA3AL1|        MH03|               REP|        6|    2019|
|RLBJKHETA3AL1|        TN01|               REP|        6|    2019|
|RLBJLHECC3AM1|        PB03|               REP|        6|    2019|
|RLBJLHER21AM1|        CGR1|               REP|        6|    2019|
|RLC3QEA623A01|        HR02|               REP|        6|    2

In [ ]:
# products_df = products_df.dropDuplicates()


In [ ]:
# products_df.show()

In [43]:
products_df = calendar_df.join(products_df,
         on=['for_month', 'for_year'],how='left_outer')

In [44]:
products_df.show()

+---------+--------+----------+-------------+-------------+------------+------------------+
|for_month|for_year|  for_date|days_in_month|     sku_code|sc_node_code|sales_channel_code|
+---------+--------+----------+-------------+-------------+------------+------------------+
|        6|    2019|2019-06-02|           30|RYQHJ0TUB0AE1|        DL04|               REP|
|        6|    2019|2019-06-02|           30|RYN5D0TUB0AE1|        DL01|               REP|
|        6|    2019|2019-06-02|           30|RYN4Q0TUB0AP1|        PY01|               REP|
|        6|    2019|2019-06-02|           30|RYN4Q0TUB0AP1|        HRR2|               REP|
|        6|    2019|2019-06-02|           30|RYN4K0TUB0AP1|        PB01|               REP|
|        6|    2019|2019-06-02|           30|RYN4G0TFT0AE1|        PB03|               REP|
|        6|    2019|2019-06-02|           30|RYN0P0TUB0AD1|        MH07|               REP|
|        6|    2019|2019-06-02|           30|RYN0P0TUB0AD1|        HRR2|        

In [45]:
products_df = products_df.join(forecast_df,
         on=['for_month', 'for_year', 'sku_code', 'sc_node_code','sales_channel_code'],
         how='left')

In [46]:
products_df = products_df.withColumn('forecast_dly', products_df['forecast_qty']/products_df['days_in_month'])

In [48]:
products_df.show()

+---------+--------+-------------+------------+------------------+----------+-------------+----------+--------+-------+------------+--------------+------------+
|for_month|for_year|     sku_code|sc_node_code|sales_channel_code|  for_date|days_in_month|source_tag|in_month|in_year|forecast_qty|forecast_value|forecast_dly|
+---------+--------+-------------+------------+------------------+----------+-------------+----------+--------+-------+------------+--------------+------------+
|        6|    2019|RYQHJ0TUB0AE1|        DL04|               REP|2019-06-02|           30|         1|       6|   2019|      0.0000|    0.00000000|         0.0|
|        6|    2019|RYN5D0TUB0AE1|        DL01|               REP|2019-06-02|           30|         1|       6|   2019|      0.0000|    0.00000000|         0.0|
|        6|    2019|RYN4Q0TUB0AP1|        PY01|               REP|2019-06-02|           30|         1|       6|   2019|      0.0000|    0.00000000|         0.0|
|        6|    2019|RYN4Q0TUB0AP1|

In [ ]:
# dbDetails = {
#     "driver": "postgresql",
#     "username": "postgres",
#     "password": "password",
#     "host": "localhost",
#     "port": "5432",
#     "database": "apollo_alpha"
# }

In [ ]:

# calendar_df = sqlContext.read \
#     .format("odbc") \
#     .option("url",'jdbc:postgresql://localhost:5432/alpha_emami') \
#     .option("dbtable", "bi.calendar_master") \
#     .load()


In [ ]:

"""
calendar_df = sqlContext.read \
    .format("odbc") \
    .option("url",'jdbc:postgresql://localhost:5432/alpha_emami') \
    .option("dbtable", "bi.calendar_master") \
    .load()
"""


In [ ]:
# products_df.show()

In [49]:
df = df.withColumnRenamed('status_date',"for_date")

In [52]:
df.show()

+----------+-------------+------------+------------------+----------+---------+
|  for_date|     sku_code|sc_node_code|sales_channel_code|stock_type|stock_qty|
+----------+-------------+------------+------------------+----------+---------+
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st1|     80.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st2|      0.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st3|      0.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st1|   2036.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st2|      0.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st3|      0.0|
|2019-06-24|RL10J0AS10AH1|        AP01|               REP|       st1|      3.0|
|2019-06-24|RL10J0AS10AH1|        AP01|               REP|       st2|      0.0|
|2019-06-24|RL10J0AS10AH1|        AP01|               REP|       st3|      0.0|
|2019-06-24|RL10J0AS10AH1|        AP02| 

In [53]:
from pyspark.sql.functions import col, unix_timestamp, to_date
df = df.withColumn('for_date', 
                   to_date(unix_timestamp(col('for_date'), 'yyyy-MM-dd').cast("timestamp")))

In [55]:
df.show()

+----------+-------------+------------+------------------+----------+---------+
|  for_date|     sku_code|sc_node_code|sales_channel_code|stock_type|stock_qty|
+----------+-------------+------------+------------------+----------+---------+
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st1|     80.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st2|      0.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st3|      0.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st1|   2036.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st2|      0.0|
|2019-06-24|RL10J0AS10AH1|        1002|               REP|       st3|      0.0|
|2019-06-24|RL10J0AS10AH1|        AP01|               REP|       st1|      3.0|
|2019-06-24|RL10J0AS10AH1|        AP01|               REP|       st2|      0.0|
|2019-06-24|RL10J0AS10AH1|        AP01|               REP|       st3|      0.0|
|2019-06-24|RL10J0AS10AH1|        AP02| 

In [56]:
final_df = products_df.join(df,
                 on=['sku_code', 'sc_node_code','sales_channel_code','for_date'],
                 how='left')

In [57]:
final_df = final_df.fillna(0)

In [68]:
%%time
final_df.write.format('com.databricks.spark.csv').save('../final_stock_secondary.csv')

Wall time: 1min 31s


In [60]:
# %%time
# final_df.write.format('csv').save('../final_stock_secondary.csv')

Wall time: 1min 57s


In [62]:
%%time
final_df.write.parquet("../proto.parquet")


Wall time: 1min 55s


In [71]:
%%time
df = sqlContext.read.parquet('../proto.parquet')
df.show()

+-------------+------------+------------------+----------+---------+--------+-------------+----------+--------+-------+------------+---------------+-------------------+----------+---------+
|     sku_code|sc_node_code|sales_channel_code|  for_date|for_month|for_year|days_in_month|source_tag|in_month|in_year|forecast_qty| forecast_value|       forecast_dly|stock_type|stock_qty|
+-------------+------------+------------------+----------+---------+--------+-------------+----------+--------+-------+------------+---------------+-------------------+----------+---------+
|RL10J0AS10AH1|        AP12|               REP|2019-06-16|        6|    2019|           30|         1|       6|   2019|     10.0000| 22500.00000000| 0.3333333333333333|      null|      0.0|
|RL10J0AS10AH1|        BR03|               REP|2019-06-02|        6|    2019|           30|         1|       6|   2019|      0.0000|     0.00000000|                0.0|      null|      0.0|
|RL10J0AS10AH1|        BR04|               REP|201

In [69]:
%%time
df = sqlContext.read.csv('../final_stock_secondary.csv')
df.show()

+-------------+----+---+----------+---+----+---+---+---+----+--------+---------------+-------------------+----+----+
|          _c0| _c1|_c2|       _c3|_c4| _c5|_c6|_c7|_c8| _c9|    _c10|           _c11|               _c12|_c13|_c14|
+-------------+----+---+----------+---+----+---+---+---+----+--------+---------------+-------------------+----+----+
|RL10J0AS10AH1|AP01|REP|2019-06-13|  6|2019| 30|  1|  6|2019|300.0000|675000.00000000|               10.0|null| 0.0|
|RL10J0AS10AH1|AP04|REP|2019-06-09|  6|2019| 30|  1|  6|2019|200.0000|450000.00000000|  6.666666666666667|null| 0.0|
|RL10J0AS10AH1|AP06|REP|2019-06-03|  6|2019| 30|  1|  6|2019| 20.0000| 45000.00000000| 0.6666666666666666|null| 0.0|
|RL10J0AS10AH1|AP07|REP|2019-06-13|  6|2019| 30|  1|  6|2019| 30.0000| 67500.00000000|                1.0|null| 0.0|
|RL10J0AS10AH1|CG03|REP|2019-06-25|  6|2019| 30|  1|  6|2019| 10.0000| 22500.00000000| 0.3333333333333333|null| 0.0|
|RL10J0AS10AH1|GA01|REP|2019-06-02|  6|2019| 30|  1|  6|2019| 12

In [ ]:
sc.stop()

In [72]:
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape
df.shape()

(4735661, 15)

TODO

-Read and Write to Database
-Make the system as master and VM's as slave and run the code
-Check is alphabet function and picking last two elements of column